In [4]:
# packages to store and manipulate data
import pandas as pd
import numpy as np

# plotting packages
import matplotlib.pyplot as plt
# model building package
import sklearn
import os
# package to clean text
import re
from datetime import datetime
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host': 'elasticsearch-covid', 'port': 9200}])

Elasticsearch tiene varios tipos de query en la que se calcula el score de diferentes maneras, hay que buscar la que mejor se ajusta al problema.

In [69]:
def serch_query_string(query,
                         index_pattern='covid*',
                         size=10,
                         fragments=0,
                         text_field='html'):
    return es.search(
        index=index_pattern,
        body={
            "from": 0, "size": size,         
            "query": {
                    "query_string" : {
                        "query" : query,
                        "default_field" : text_field,
                        "minimum_should_match": 1
                    }
                },
             "highlight": {
                "fields": {
                    "*": {}
                },
                "number_of_fragments": fragments,
                 "fragment_size": 500,
                 "pre_tags":["<strong><u>"],
                 "post_tags":["</u></strong>"]
            }
        })['hits']['hits']

In [74]:
from IPython.core.display import display, HTML
def get(fragments,text):
    for result in serch_query_string(text,fragments=fragments):
        print('Score: {}'.format(result['_score']))
        for fragment in result['highlight']['html']:
            display(HTML(fragment))
            print('__')
        print()

Además del score puede indicar donde hizo el match. Puede develver cierto número de fragmentos que contienen el match o el documento completo

In [76]:
get(3,'¿Cuánto demora el proceso de registro del test desarrollado por Udelar y el Institut Pasteur?')

Score: 30.528679


__


__


__

Score: 20.767653


__


...

__


__

Score: 8.554329


__


__


__



In [77]:
get(0,'¿Cuánto demora el proceso de registro del test desarrollado por Udelar y el Institut Pasteur?')

Score: 30.528679


__

Score: 20.767653


__

Score: 19.442581


__

Score: 18.45873


__

Score: 10.97562


...

__

Score: 10.031465


__

Score: 9.740364


__

Score: 9.19537


__

Score: 8.554329


__



In [78]:
get(0,'¿Cuántos días habrían pasado desde que se iniciaron las medidas de aislamiento si se levantan después de Turismo?')

Score: 9.967554


__

Score: 9.24989


__

Score: 9.0345


__

Score: 8.0709305


__

Score: 8.046806


...

__

Score: 7.7043695


__

Score: 7.461256


__

Score: 7.3158164


__

Score: 7.117929


__

